In [ ]:
import os
import cv2
import cvzone
import numpy as np
from cvzone.PoseModule import PoseDetector

# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Width
cap.set(4, 720)   # Height

# Initialize pose detector
detector = PoseDetector()

# Load shirt images
shirtFolderPath = "Resources/Shirts"
listShirts = os.listdir(shirtFolderPath)

if not listShirts:
    raise Exception("❌ No shirt images found in 'Resources/Shirts'!")

# Constants
scalingFactor = 3.2  # Controls overall size, increase if too small
smoothingFactor = 0.85  # Higher value means smoother motion
shirtRatioHeightWidth = 1.5  # Fine-tuned ratio
imageNumber = 1 # Default shirt index

# Initialize smoothing variables
prev_x_offset, prev_y_offset = 0, 0
prev_widthOfShirt, prev_heightOfShirt = 100, 160  # Default reasonable size
prev_shoulder_width = 100  # For dynamic scaling

while True:
    success, img = cap.read()
    if not success:
        print("❌ Error: Could not read frame from webcam.")
        break  # Exit if no frame is read

    img = detector.findPose(img)
    lmList, bboxInfo = detector.findPosition(img, bboxWithHands=False, draw=False)

    if lmList:
        # Get shoulder landmarks
        left_shoulder = np.array(lmList[11][1:3])  # (x, y) Left Shoulder
        right_shoulder = np.array(lmList[12][1:3])  # (x, y) Right Shoulder

        # Compute midpoint & shoulder width
        mid_shoulder = (left_shoulder + right_shoulder) // 2
        shoulder_width = np.linalg.norm(left_shoulder - right_shoulder)  # Distance between shoulders

        # Apply smoothing to shoulder width to prevent rapid scaling
        shoulder_width = prev_shoulder_width * smoothingFactor + shoulder_width * (1 - smoothingFactor)
        prev_shoulder_width = shoulder_width

        # Load shirt image
        shirtPath = os.path.join(shirtFolderPath, listShirts[imageNumber])
        imgShirt = cv2.imread(shirtPath, cv2.IMREAD_UNCHANGED)
        if imgShirt is None:
            print(f"❌ Error: Could not load {shirtPath}")
            continue

        # Compute width & height of shirt using shoulder width
        widthOfShirt = int(shoulder_width * scalingFactor)
        heightOfShirt = int(widthOfShirt * shirtRatioHeightWidth)
        
        # Apply EMA smoothing to width & height
        widthOfShirt = int(prev_widthOfShirt * smoothingFactor + widthOfShirt * (1 - smoothingFactor))
        heightOfShirt = int(prev_heightOfShirt * smoothingFactor + heightOfShirt * (1 - smoothingFactor))

        prev_widthOfShirt, prev_heightOfShirt = widthOfShirt, heightOfShirt

        # Resize shirt
        imgShirt = cv2.resize(imgShirt, (widthOfShirt, heightOfShirt))

        # Adjust position to align with shoulders dynamically
        x_offset = int((mid_shoulder[0] - widthOfShirt // 2)+ 420)
        y_offset = int(mid_shoulder[1] + 300)  # Adjusted for better alignment

        # Apply EMA smoothing to position
        x_offset = int(prev_x_offset * smoothingFactor + x_offset * (1 - smoothingFactor))
        y_offset = int(prev_y_offset * smoothingFactor + y_offset * (1 - smoothingFactor))

        prev_x_offset, prev_y_offset = x_offset, y_offset

        # Overlay the shirt
        try:
            img = cvzone.overlayPNG(img, imgShirt, (x_offset, y_offset))
        except Exception as e:
            print(f"❌ Overlay error: {e}")

    # Display output
    cv2.imshow("Virtual Try-On", img)
    cv2.waitKey(1)
